In [ ]:
import numpy as np
import pandas as pd


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")

import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from plotly import graph_objs as go

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vishn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vishn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vishn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:

df = pd.read_csv('drugsComTrain_raw.tsv', delimiter='\t')

df.head()


,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [ ]:
df['condition'].value_counts()

condition
Birth Control                                                          28788
Depression                                                              9069
Pain                                                                    6145
Anxiety                                                                 5904
Acne                                                                    5588
Bipolar Disorde                                                         4224
Insomnia                                                                3673
Weight Loss                                                             3609
Obesity                                                                 3568
ADHD                                                                    3383
Diabetes, Type 2                                                        2554
Emergency Contraception                                                 2463
High Blood Pressure                                               

In [ ]:
df_train = df[(df['condition']=='Birth Control') | (df['condition']=='Depression') | (df['condition']=='High Blood Pressure')|(df['condition']=='Diabetes, Type 2')| (df['condition']=='Pain') | (df['condition']=='Anxiety') | (df['condition']=='Insomnia') | (df['condition']=='Obesity') |(df['condition']=='ADHD') |(df['condition']=='Emergency Contraception') |(df['condition']=='Abnormal Uterine Bleeding ')|(df['condition']=='Bowel Preparation')|(df['condition']=='Smoking Cessation')|(df['condition']=='Migraine')|(df['condition']=='Constipation')  ]


In [ ]:
df_train['condition'].value_counts()

condition
Birth Control              28788
Depression                  9069
Pain                        6145
Anxiety                     5904
Insomnia                    3673
Obesity                     3568
ADHD                        3383
Diabetes, Type 2            2554
Emergency Contraception     2463
High Blood Pressure         2321
Bowel Preparation           1859
Smoking Cessation           1780
Migraine                    1694
Constipation                1595
Name: count, dtype: int64

In [ ]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    text = text.strip()
    return text

In [ ]:
def preprocess_text(text):
    text = clean_text(text)
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

In [ ]:
df_train['preprocessed_text'] = df_train['review'].apply(preprocess_text)


In [ ]:
df_train.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,preprocessed_text
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,son halfway fourth week intuniv became concern...
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,used take another oral contraceptive pill cycl...
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,first time using form birth control im glad we...
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1.0,"March 7, 2017",5,pulled cummed bit took plan b hour later took ...
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8.0,"December 8, 2016",1,pill many year doctor changed rx chateal effec...


In [ ]:
df_train.shape

(74796, 8)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
df_train['condition_label'] = label_encoder.fit_transform(df_train['condition'])

num_classes = len(label_encoder.classes_)

y = to_categorical(df_train['condition_label'], num_classes=num_classes)


# LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


max_words = 10000
max_seq_length = 100
embedding_dim = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_train['preprocessed_text'])
sequences = tokenizer.texts_to_sequences(df_train['preprocessed_text'])
X = pad_sequences(sequences, maxlen=max_seq_length)
y = df_train['condition_label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim))
model.add(SpatialDropout1D(0.2))  # adding dropout layer to avoid overfitting
model.add(LSTM(units=128))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))




Epoch 1/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 266s 138ms/step - accuracy: 0.6125 - loss: 1.2241 - val_accuracy: 0.8787 - val_loss: 0.4089
Epoch 2/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 262s 140ms/step - accuracy: 0.8956 - loss: 0.3447 - val_accuracy: 0.9027 - val_loss: 0.3113
Epoch 3/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 271s 145ms/step - accuracy: 0.9263 - loss: 0.2319 - val_accuracy: 0.9195 - val_loss: 0.2681
Epoch 4/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 278s 148ms/step - accuracy: 0.9450 - loss: 0.1791 - val_accuracy: 0.9217 - val_loss: 0.2599
Epoch 5/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 282s 151ms/step - accuracy: 0.9542 - loss: 0.1441 - val_accuracy: 0.9230 - val_loss: 0.2619
Epoch 6/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 289s 155ms/step - accuracy: 0.9656 - loss: 0.1150 - val_accuracy: 0.9269 - val_loss: 0.2610
Epoch 7/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 286s 153ms/step - accuracy: 0.9721 - loss: 0.0896 - val_accuracy: 0.9297 - val_loss: 0.2702
Epoch 8/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 285s 153ms/step - ac

In [ ]:
from sklearn.metrics import classification_report

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

# Predict classes for the test set
y_pred_lstm = np.argmax(model.predict(X_val), axis=-1)

# Print classification report
print("Classification Report:")
print(classification_report(y_val, y_pred_lstm))


468/468 ━━━━━━━━━━━━━━━━━━━━ 22s 47ms/step - accuracy: 0.9315 - loss: 0.2936
Validation Loss: 0.3007316291332245, Validation Accuracy: 0.9332219362258911
468/468 ━━━━━━━━━━━━━━━━━━━━ 22s 46ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.91      0.93       655
           1       0.83      0.77      0.80      1125
           2       0.99      0.99      0.99      5762
           3       0.97      0.98      0.97       373
           4       0.92      0.91      0.91       301
           5       0.85      0.88      0.86      1846
           6       0.92      0.93      0.92       498
           7       0.99      0.95      0.97       518
           8       0.92      0.86      0.89       465
           9       0.85      0.88      0.87       676
          10       0.89      0.92      0.90       349
          11       0.94      0.94      0.94       741
          12       0.92      0.91      0.92      1252
          13       0.98

In [ ]:
# Save the LSTM model
model.save("lstm_model.h5")


In [ ]:
# Preprocess the new text
new_text = [df_train['preprocessed_text'][15]]


new_sequence = tokenizer.texts_to_sequences(new_text)

# Pad the sequence
padded_new_sequence = pad_sequences(new_sequence, maxlen=max_seq_length)

# Use the model to predict
prediction = model.predict(padded_new_sequence)
predicted_class = np.argmax(prediction)


label_to_condition_mapping = dict(zip(df_train['condition_label'], df_train['condition']))

predicted_class_label = label_to_condition_mapping[predicted_class]

# Print the predicted class label
print("Predicted class:", predicted_class_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted class: Pain


In [ ]:


lstm = confusion_matrix(y_val, y_pred_lstm)
lstm

array([[ 606,    8,    2,    0,    0,   24,    4,    1,    3,    2,    1,
           1,    3,    0],
       [  16,  908,    5,    0,    0,  150,    1,    1,    4,   15,    1,
           2,   21,    1],
       [   7,    2, 5726,    0,    1,    7,    4,    4,    2,    1,    1,
           2,    5,    0],
       [   0,    1,    1,  365,    5,    1,    0,    0,    0,    0,    0,
           0,    0,    0],
       [   0,    2,    5,    4,  274,    2,    2,    1,    0,    1,    2,
           0,    8,    0],
       [  32,  110,   27,    1,    3, 1614,    5,    1,    9,   16,    2,
           6,   17,    3],
       [   0,    4,    3,    2,    0,    5,  461,    1,    5,    0,    0,
          10,    7,    0],
       [   0,    0,   13,    0,    1,    1,    0,  499,    0,    2,    1,
           0,    1,    0],
       [   5,   10,    6,    0,    2,   12,    4,    1,  400,    3,    4,
           5,   13,    0],
       [   7,   35,    5,    1,    1,   38,    1,    0,   10,  551,    3,
           1,   2

# CNN

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense


cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=max_words, output_dim=embedding_dim))
cnn_model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(units=num_classes, activation='softmax'))

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the CNN model


Epoch 1/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.7122 - loss: 0.9530 - val_accuracy: 0.9122 - val_loss: 0.2760
Epoch 2/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9388 - loss: 0.2016 - val_accuracy: 0.9299 - val_loss: 0.2317
Epoch 3/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9725 - loss: 0.0997 - val_accuracy: 0.9366 - val_loss: 0.2259
Epoch 4/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9883 - loss: 0.0480 - val_accuracy: 0.9386 - val_loss: 0.2369
Epoch 5/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9935 - loss: 0.0269 - val_accuracy: 0.9396 - val_loss: 0.2688
Epoch 6/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9959 - loss: 0.0164 - val_accuracy: 0.9382 - val_loss: 0.2913
Epoch 7/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9962 - loss: 0.0128 - val_accuracy: 0.9394 - val_loss: 0.3271
Epoch 8/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9969 - loss: 0

In [ ]:
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_val, y_val)
print(f'CNN Validation Loss: {cnn_loss}, Validation Accuracy: {cnn_accuracy}')

# Predict classes for the test set
y_pred_cnn = np.argmax(cnn_model.predict(X_val), axis=-1)

# Print classification report
print("Classification Report:")
print(classification_report(y_val, y_pred_cnn))


468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9359 - loss: 0.3993
CNN Validation Loss: 0.39981088042259216, Validation Accuracy: 0.9352272748947144
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       655
           1       0.84      0.81      0.83      1125
           2       0.99      0.99      0.99      5762
           3       0.95      0.98      0.96       373
           4       0.91      0.90      0.91       301
           5       0.86      0.89      0.87      1846
           6       0.96      0.92      0.94       498
           7       0.99      0.96      0.98       518
           8       0.91      0.86      0.88       465
           9       0.88      0.84      0.86       676
          10       0.93      0.87      0.90       349
          11       0.92      0.96      0.94       741
          12       0.90      0.91      0.91      1252
          13       0.9

In [ ]:
cnn_model.save("cnn_model.h5")


In [ ]:
# Preprocess the new text
new_text = [df_train['preprocessed_text'][16]]

# Tokenize the preprocessed text
new_sequence = tokenizer.texts_to_sequences(new_text)

# Pad the sequence
padded_new_sequence = pad_sequences(new_sequence, maxlen=max_seq_length)

# Use the model to predict
prediction = cnn_model.predict(padded_new_sequence)
predicted_class = np.argmax(prediction)


label_to_condition_mapping = dict(zip(df_train['condition_label'], df_train['condition']))

predicted_class_label = label_to_condition_mapping[predicted_class]

# Print the predicted class label
print("Predicted class:", predicted_class_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Predicted class: Bowel Preparation


In [ ]:


cnn = confusion_matrix(y_val, y_pred_cnn)
cnn

array([[ 600,   12,    0,    1,    0,   22,    3,    0,    4,    8,    1,
           3,    1,    0],
       [   5,  915,    6,    1,    0,  152,    0,    0,    2,   17,    2,
           3,   21,    1],
       [   1,    4, 5723,    0,    1,    6,    2,    5,    1,    1,    1,
           7,   10,    0],
       [   0,    1,    0,  365,    4,    0,    0,    0,    0,    1,    0,
           2,    0,    0],
       [   0,    1,    2,   11,  272,    1,    1,    0,    1,    2,    0,
           3,    7,    0],
       [  19,   76,   18,    0,    4, 1650,    4,    0,   11,   28,    1,
          13,   17,    5],
       [   3,    4,    3,    1,    2,    1,  457,    0,    3,    0,    2,
          14,    8,    0],
       [   0,    1,   13,    0,    1,    2,    0,  499,    0,    2,    0,
           0,    0,    0],
       [   5,    8,    2,    1,    4,   11,    4,    0,  398,    3,    5,
           5,   19,    0],
       [   9,   34,    2,    1,    2,   38,    0,    0,    6,  568,    0,
           2,   1

# BI-LSTM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have defined max_words, max_seq_length, and num_classes earlier
max_words = 10000
max_seq_length = 100
embedding_dim = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_train['preprocessed_text'])
sequences = tokenizer.texts_to_sequences(df_train['preprocessed_text'])
X = pad_sequences(sequences, maxlen=max_seq_length)
y = df_train['condition_label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the BiLSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim))
model.add(SpatialDropout1D(0.2))  # adding dropout layer to avoid overfitting
model.add(Bidirectional(LSTM(units=128)))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 157s 82ms/step - accuracy: 0.5579 - loss: 1.3593 - val_accuracy: 0.8666 - val_loss: 0.4498
Epoch 2/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 154s 82ms/step - accuracy: 0.8894 - loss: 0.3587 - val_accuracy: 0.9100 - val_loss: 0.2951
Epoch 3/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 148s 79ms/step - accuracy: 0.9277 - loss: 0.2329 - val_accuracy: 0.9173 - val_loss: 0.2709
Epoch 4/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 135s 72ms/step - accuracy: 0.9446 - loss: 0.1814 - val_accuracy: 0.9164 - val_loss: 0.2815
Epoch 5/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 138s 74ms/step - accuracy: 0.9543 - loss: 0.1463 - val_accuracy: 0.9252 - val_loss: 0.2542
Epoch 6/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 114s 61ms/step - accuracy: 0.9654 - loss: 0.1102 - val_accuracy: 0.9277 - val_loss: 0.2659
Epoch 7/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 120s 64ms/step - accuracy: 0.9728 - loss: 0.0900 - val_accuracy: 0.9261 - val_loss: 0.2844
Epoch 8/10
1870/1870 ━━━━━━━━━━━━━━━━━━━━ 139s 74ms/step - accuracy: 

In [ ]:
biLstm_loss, bi_lstm = model.evaluate(X_val, y_val)
print(f'Bilstm Validation Loss: {cnn_loss}, Validation Accuracy: {cnn_accuracy}')

# Predict classes for the test set
y_pred_bi_lstm = np.argmax(model.predict(X_val), axis=-1)

# Print classification report
print("Classification Report:")
print(classification_report(y_val, y_pred_bi_lstm))


468/468 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - accuracy: 0.9330 - loss: 0.3076
Bilstm Validation Loss: 0.39981088042259216, Validation Accuracy: 0.9352272748947144
468/468 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       655
           1       0.82      0.81      0.81      1125
           2       0.99      0.99      0.99      5762
           3       0.98      0.98      0.98       373
           4       0.93      0.91      0.92       301
           5       0.85      0.87      0.86      1846
           6       0.92      0.93      0.92       498
           7       0.98      0.96      0.97       518
           8       0.91      0.86      0.88       465
           9       0.93      0.82      0.87       676
          10       0.91      0.89      0.90       349
          11       0.95      0.94      0.94       741
          12       0.91      0.93      0.92      1252
          13   

In [ ]:
# Preprocess the new text
new_text = [df_train['preprocessed_text'][25]]

# Tokenize the preprocessed text
new_sequence = tokenizer.texts_to_sequences(new_text)

# Pad the sequence
padded_new_sequence = pad_sequences(new_sequence, maxlen=max_seq_length)

# Use the model to predict
prediction = cnn_model.predict(padded_new_sequence)
predicted_class = np.argmax(prediction)


label_to_condition_mapping = dict(zip(df_train['condition_label'], df_train['condition']))

predicted_class_label = label_to_condition_mapping[predicted_class]

# Print the predicted class label
print("Predicted class:", predicted_class_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: Birth Control


In [ ]:


bilstm = confusion_matrix(y_val, y_pred_bi_lstm)
bilstm

array([[ 606,    8,    2,    0,    0,   24,    4,    1,    3,    2,    1,
           1,    3,    0],
       [  16,  908,    5,    0,    0,  150,    1,    1,    4,   15,    1,
           2,   21,    1],
       [   7,    2, 5726,    0,    1,    7,    4,    4,    2,    1,    1,
           2,    5,    0],
       [   0,    1,    1,  365,    5,    1,    0,    0,    0,    0,    0,
           0,    0,    0],
       [   0,    2,    5,    4,  274,    2,    2,    1,    0,    1,    2,
           0,    8,    0],
       [  32,  110,   27,    1,    3, 1614,    5,    1,    9,   16,    2,
           6,   17,    3],
       [   0,    4,    3,    2,    0,    5,  461,    1,    5,    0,    0,
          10,    7,    0],
       [   0,    0,   13,    0,    1,    1,    0,  499,    0,    2,    1,
           0,    1,    0],
       [   5,   10,    6,    0,    2,   12,    4,    1,  400,    3,    4,
           5,   13,    0],
       [   7,   35,    5,    1,    1,   38,    1,    0,   10,  551,    3,
           1,   2

**Comprehensive Error Analysis Report**
--

### Overview:

This report provides a detailed analysis of the performance of LSTM, CNN, and BiLSTM models for a classification task. We evaluate the models using various metrics such as accuracy, precision, recall, F1-score, and confusion matrices to gain insights into their strengths, weaknesses, and areas for improvement.

---

### Model Performance:

#### LSTM Model:
- **Accuracy**: The LSTM model achieved an overall accuracy of approximately 93.32%.
- **Precision**: Precision values range from 0.83 to 0.99, indicating high precision in most classes.
- **Recall**: Recall rates vary between 0.77 and 0.98, with some variability across classes.
- **F1-score**: F1-scores are high, ranging from 0.80 to 0.99, reflecting a good balance between precision and recall.
- **Confusion Matrix Analysis**:
  - Most common misclassifications occur between classes 1, 5, and 11, despite the high accuracy.
  - Class 1 (precision: 0.83, recall: 0.77) and Class 5 (precision: 0.85, recall: 0.88) exhibit lower precision comparedve model performance.

#### CNN Model:
- **Accuracy**: The CNN model achieved an overall accuracy of approximately 93.52%.
- **Precision**: Precision values range from 0.84 to 0.99, indicating high precision in most classes.
- **Recall**: Recall rates vary between 0.81 and 0.96, with some variability across classes.
- **F1-score**: F1-scores are high, ranging from 0.83 to 0.99, reflecting a good balance between precision and recall.
- **Confusion Matrix Analysis**:
  - Similar to the LSTM model, frequent misclassifications occur between classes 1, 5, and 11, despite the high accuracy.
  - Class 1 (precision: 0.84, recall: 0.81) and Class 5 (precision: 0.86, recall: 0.89) exhibit lower precisiish between challenging classes.

#### BiLSTM Model:
- **Accuracy**: The BiLSTM model achieved an overall accuracy of approximately 93.35%.
- **Precision**: Precision values range from 0.82 to 0.99, indicating high precision in most classes.
- **Recall**: Recall rates vary between 0.81 and 0.94, with some variability across classes.
- **F1-score**: F1-scores are high, ranging from 0.81 to 0.99, reflecting a good balance between precision and recall.
- **Confusion Matrix Analysis**:
  - Similar to the LSTM and CNN models, frequent misclassifications occur between classes 1, 5, and 11, despite the high accuracy.
  - Class 1 (precision: 0.82, recall: 0.81) and Class 5 (precision: 0.85, recall: 0.87) exhibit lorobustness and mitigate misclassifications.

---

### General Observations:

- **Common Error Patterns**: Across all models, classes 1, 5, and 11 exhibit consistent misclassifications, indicating inherent challenges in distinguishing between these classes.
- **Performance Discrepancies**: While overall accuracies are high, specific classes experience lower precision and recall, highlighting areas for targeted improvement.
- **Model Consistency**: Despite differences in architecture (LSTM, CNN, BiLSTM), the error patterns remain consistent, suggesting undeclass differentiation and overall model performance.

---

### Conclusion:

In conclusion, while the LSTM, CNN, and BiLSTM models demonstrate high accuracy and performance across various metrics, they exhibit consist' r improvement, and offers suggestions for further enhancement.ents in model performance.